In [ ]:
import sys
import json
sys.path.insert(19, 'litepose-pose-estimation/src')
import torch
from torchvision import transforms
import lp_config.lp_common_config as cc
from lp_model.lp_litepose import LitePose
from lp_inference.lp_inference import inference, assocEmbedding
from lp_utils.lp_image_processing import drawHeatmap, drawKeypoints, drawSkeleton
import cv2
from PIL import Image
from datetime import datetime
import numpy as np
import pandas as pd
import imutils
sys.path.insert(20, 'src/referee_gloves_detector')
from color_labeler import ColorLabeler
import matplotlib.pyplot as plt

In [ ]:
# Leggo il file di configurazione
with open('config/config.json') as f:
    config_data = json.load(f)

# Ottengo i percorsi dei file
file_path_big_arch = config_data['path_big_arch']
file_path_csv_keypoints_webcam = config_data['path_csv_keypoints_webcam']

In [ ]:
model = LitePose().to(cc.config["device"])
model.load_state_dict(torch.load(file_path_big_arch, map_location=cc.config["device"]))

In [ ]:
# Red Filtering

image = cv2.imread('C:\\Users\\stolf\\dev\\Progetto Visione e Percezione\\src\\referee_gloves_detector\\bloisi_nao_red_hands.png')
cv2.imshow("Original", image)
 
result = image.copy()
 
image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
 
lower1 = np.array([0, 100, 100])
upper1 = np.array([5, 255, 255])
 
lower2 = np.array([174,100,100])
upper2 = np.array([179,255,255])
 
lower_mask = cv2.inRange(image, lower1, upper1)
upper_mask = cv2.inRange(image, lower2, upper2)
 
full_mask = lower_mask + upper_mask;
 
result = cv2.bitwise_and(result, result, mask=full_mask)
 
cv2.imshow('mask', full_mask)
cv2.imshow('result', result)
 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Segmentation
# TODO: migliorare o togliere del tutto, infondo a cosa dovrebbe servirci?

# Converto l'immagine in grayscale
gray_image = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
 
# Converto l'immagine da grayscale a binaria
ret,thresh = cv2.threshold(gray_image,127,255,0)
 
# Trovo i contorni nell'immagine binaria
contours, im2 = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
for c in contours:
    # Calcolo il centroide
    M = cv2.moments(c)
 
    # Calcolo le coordinate x e y del centroide
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
    else:
        cX, cY = 0, 0
    cv2.circle(result, (cX, cY), 5, (255, 255, 255), -1)
    cv2.putText(result, "centroid", (cX - 25, cY - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    cv2.imshow("Image", result)
    cv2.waitKey(0)

In [ ]:
# Cropping

def leftmost_red_pixel(numpy_image):

    image = Image.fromarray(numpy_image.astype('uint8'), 'RGB')
    width, height = image.size

    for x in range(width):
        for y in range(height):
            r, g, b = image.getpixel((x, y))

            if r > 0:
                return (x, y)

    return (None,None)

def rightmost_red_pixel(numpy_image):

    image = Image.fromarray(numpy_image.astype('uint8'), 'RGB')
    width, height = image.size

    for x in range(width - 1, -1, -1):
        for y in range(height):
            r, g, b = image.getpixel((x, y))

            if r > 0:
                return (x, y)

    return (None,None)

print(result.shape)
im_pil_1 = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
(xs,ys) = leftmost_red_pixel(result)
(xd,yd) = rightmost_red_pixel(result)
if(xs != None and ys != None and (ys-10)>0):
    col = ys-10
    red_filtered_image_restricted = result[:,col:,:]
    print(red_filtered_image_restricted.shape)
if(xd != None and yd != None and (yd+10)<(np.shape(red_filtered_image_restricted)[1])):
    col = yd+10
    red_filtered_image_restricted = red_filtered_image_restricted[:,:col,:]
    print(red_filtered_image_restricted.shape)
im_pil_2 = cv2.cvtColor(red_filtered_image_restricted, cv2.COLOR_BGR2RGB)
f, axarr = plt.subplots(nrows=1, ncols=2)
axarr[0].imshow(im_pil_1)
axarr[1].imshow(im_pil_2)

In [ ]:
# uri = "C:\\Users\\stolf\\dev\\Progetto Visione e Percezione\\src\\referee_gloves_detector\\example_shapes.png"
# uri = "C:\\Users\\stolf\\dev\\Progetto Visione e Percezione\\src\\referee_gloves_detector\\example_shapes_1.png"
# uri = "C:\\Users\\stolf\\dev\\Progetto Visione e Percezione\\src\\referee_gloves_detector\\example_shapes_2.png"
uri = "C:\\Users\\stolf\\dev\\Progetto Visione e Percezione\\src\\referee_gloves_detector\\bloisi_nao_red_hands.png"

In [ ]:
# Apro la webcam e salvo al volo i frame e il timestamp

webcam=cv2.VideoCapture(0) 

if not webcam.isOpened():
    raise Exception("Errore nell'apertura della webcam")

keypoints_vec = []
timestamps = []
resize = transforms.Resize([224, 224])  
to_tensor = transforms.ToTensor()
ret,frame=webcam.read()
while ret:
    
    
    istante_attuale = datetime.now()
    stringa_istante = istante_attuale.strftime("%d/%m/%Y %H:%M:%S.%f")
    timestamps.append(stringa_istante) 
    
    frame = cv2.flip(frame, 1)
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = img[:,80:]
    img = img[:,:-80]
    

    im_pil = Image.fromarray(img)
    frame = resize(im_pil)
    tensor = to_tensor(frame)
    tensor = tensor.unsqueeze(0)
    
    
    if ret==True:
        output, keypoints = inference(model, tensor)
        keypoints_vec.append(keypoints)
        embedding = assocEmbedding(keypoints)
        frame_modified = drawSkeleton(tensor[0], embedding[0])
        #frame = drawKeypoints(tensor[0], keypoints[0])
        cv2.imshow("Pose estimation", frame_modified)
        key=cv2.waitKey(1) & 0xFF
        if key==ord("q"):
            break         
    ret, frame = webcam.read()
webcam.release()
cv2.destroyAllWindows()
print("Dimensioni dell'array di frame:", len(keypoints_vec))
print("Dimensioni dell'array di timestamp:", len(timestamps))

In [ ]:
# Salvo solo i 9 keypoints scelti per ogni frame

d = {'timestamp':[],'x': [], 'y': [], 'tag':[]}
df_keypoints = pd.DataFrame(data=d)
for timestamp, restricted_keypoints in zip(timestamps, keypoints_vec):
    for i in range(len(restricted_keypoints[0])):
        if not (restricted_keypoints[0][i]):
            df_temp = pd.DataFrame([{'timestamp': '01/01/1970 01:00:00','x':np.nan, 'y':np.nan, 'tag':np.nan}])
            df_keypoints = pd.concat([df_keypoints, df_temp])
        else:
            df_temp = pd.DataFrame(restricted_keypoints[0][i])
            df_temp['timestamp'] = timestamp
            df_keypoints = pd.concat([df_keypoints, df_temp])
df_keypoints.to_csv(file_path_csv_keypoints_webcam, index=False)
df_multiindex = df_keypoints.set_index(['timestamp'])
display(df_multiindex)